<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & setup

In [0]:
import numpy as np
import pandas as pd

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn import metrics,model_selection

from collections import OrderedDict
import string, json

In [0]:
#Online instance only.
from google.colab import drive #drive.flush_and_unmount() #drive.flush()
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')
nltk.download('all',quiet=True)
!nvidia-smi

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Thu Nov 28 01:41:35 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================

# Cleaning Functions

In [0]:
# List of all words to remove from statements
removables = []
removables += nltk.corpus.stopwords.words('english')
#removables += [w.lower() for w in nltk.corpus.names.words()]
removables += ['hi','day','go']
replacements = {}
replacements = OrderedDict({'barack obama':'obama','donald trump':'trump','health care':'healthcare','hillary clinton':'hillary','president obama':'obama','barack':'obama','donald':'trump','gov ':'government ','said':'say'})
lemmatizer = nltk.stem.WordNetLemmatizer()

In [0]:
#much info was lost
#some of these could be specified as options to the Tfidf vectorizer instead
def filterSent(s):
    s = s.lower()
    for rep,val in replacements.items(): s = s.replace(rep,val)
    s = nltk.word_tokenize(s) #split into list of words intelligently
    s = tuple(w for w in s if w not in removables) #remove removables
    s = tuple(lemmatizer.lemmatize(w) for w in s if w.isalpha()) #supposed to lemmatize words (aka get basic form)
    return ' '.join(s)
filterSent("hi there my name is jOhn henry battery car dead better programming")

#applied to dataset to reformat data
formatter = lambda x:(filterSent(x.statement),liar_score[x.label])

def printScore(y_pred,y_test):
    #assuming balanced, otherwise using balanced_accuracy_score()
    print("Accuracy: {0:.1%}".format(metrics.accuracy_score(y_test, y_pred)))
    #TODO add more

# LIAR dataset

In [0]:
path_to_liar = './raw/liar_dataset/'
#counts refer to the speaker's reliability as a whole, interestingly excluding true counts. 
liar_headers = ('id','label','statement',
                'subjects','speaker','job_title',
                'state','party','barely_true_counts',
                'false_counts','half_true_counts','mostly_true_counts',
                'pants_on_fire_counts','context')

liar_score = {'true':1,
              'mostly-true':1,
              'half-true':1,
              'barely-true':0,
              'false':0,
              'pants-fire':0
             }

In [0]:
#Opens dataset, extracts only statement & label and reformats them using formatter()
liar_train = pd.read_csv(path_to_liar+'train.tsv',delimiter='\t',header=None,names=liar_headers)
liar_train.loc[:,('statement','label')] = liar_train.loc[:,('statement','label')].apply(formatter,axis=1,result_type='broadcast')
liar_test = pd.read_csv(path_to_liar+'test.tsv',delimiter='\t',header=None,names=liar_headers)
liar_test.loc[:,('statement','label')] = liar_test.loc[:,('statement','label')].apply(formatter,axis=1,result_type='broadcast')

In [0]:
minlen = 5
maxlen = 30
liar_train = liar_train[liar_train.statement.apply(lambda x:minlen<=len(x.split())<=maxlen)]

# Bag of Words + SVM approach

In [0]:
#data fed to vectorizer is already cleaned but maybe TODO use their params instead
vectorizer = TfidfVectorizer(max_features=200,ngram_range=(1,2)) #gets kwords on principle of very used in doc, but rare across all docs
#vectorizer = CountVectorizer(max_features=400) #gets kwords by frequency distribution (is not smart)

In [0]:
#trains vectorizer on training dataset, followed by transforming training dataset into vectors
train_data = vectorizer.fit_transform(liar_train.statement)
x_train = train_data.toarray()
y_train = np.array(liar_train.label,dtype=int)
#transforms test dataset into vectors without training
test_data = vectorizer.transform(liar_test.statement)
x_test = test_data.toarray()
y_test = np.array(liar_test.label,dtype=int)
#see which keywords were extracted as features
print(vectorizer.get_feature_names())

['abortion', 'act', 'actually', 'administration', 'almost', 'america', 'american', 'average', 'back', 'benefit', 'bill', 'billion', 'border', 'budget', 'bush', 'business', 'campaign', 'candidate', 'care', 'child', 'city', 'clinton', 'college', 'come', 'company', 'congress', 'cost', 'could', 'country', 'county', 'court', 'created', 'crime', 'cut', 'debt', 'deficit', 'democrat', 'democratic', 'department', 'district', 'dollar', 'dont', 'economy', 'education', 'election', 'employee', 'even', 'every', 'family', 'federal', 'first', 'five', 'florida', 'four', 'fund', 'funding', 'georgia', 'get', 'give', 'going', 'got', 'gov', 'government', 'governor', 'gun', 'half', 'health', 'health insurance', 'healthcare', 'high', 'higher', 'highest', 'hillary', 'history', 'home', 'house', 'illegal', 'immigrant', 'immigration', 'income', 'increase', 'insurance', 'island', 'job', 'john', 'know', 'last', 'last year', 'law', 'le', 'legislation', 'like', 'lost', 'made', 'majority', 'make', 'many', 'mccain', '

In [0]:
#fits SVM on vectorized training dataset
clf = SVC(kernel='linear')
clf.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = clf.predict(x_test)
printScore(y_pred,y_test)

Accuracy: 60.1%


## Analysis
Bag of Words + SVM is essentially classifying fake news by similarity to other fake news via keywords. Many of these keywords picked by tfidf vectorizer are names and terms that are often engaged with controversial topics, likely due to the political nature of the dataset. Likewise, since fake news tends to surround controversial topics, controversial keywords end up being the main features used by SVM to classify even though they don't entail being fake. Unfortunately, lemmatization doesn't seem to fix things like "theyre" or "cuban" vs "cuba".

# ELMo Sum + SVM approach

!pip install allennlp

In [0]:
from allennlp.modules.elmo import Elmo, batch_to_ids
from torch import nn
from sklearn.decomposition import PCA
options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 1, dropout=0).to('cuda') #1 representation only

In [0]:
batch_size = 32
def getEmbedded(df):
    batches = np.array_split(df,int(len(df)/batch_size))
    edf = pd.DataFrame({})
    for batch in batches:
        sentences = batch.statement.str.split()
        character_ids = batch_to_ids(sentences).to('cuda')
        embeddings =  elmo(character_ids)['elmo_representations'] #[representation][sent_no][word_no (padded)]
        normalized = nn.functional.softmax(embeddings[0].sum(1),dim=1).to('cpu').detach()
        embed = pd.DataFrame(normalized.numpy())
        embed = embed.assign(label=batch.label.values)
        edf = edf.append(embed,ignore_index=True)
    return edf

In [0]:
train_df = getEmbedded(liar_train)
test_df = getEmbedded(liar_test)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,label
0,3.575464e-07,3.161658e-07,5.078559e-08,2.582136e-08,9.453494e-08,1.321333e-05,2.250624e-08,8.324159e-07,8.880279e-08,1.889627e-07,4.245820e-08,6.092341e-06,2.380327e-08,2.220460e-07,6.266293e-08,2.327942e-08,3.523541e-07,6.563824e-08,7.673962e-08,1.422934e-08,1.361767e-06,2.284145e-07,3.371377e-08,1.785008e-08,7.911170e-07,5.995866e-07,9.646606e-08,8.656997e-08,6.128693e-08,4.203974e-08,5.419511e-07,1.006815e-07,7.163814e-08,3.863121e-08,2.674054e-08,5.975621e-08,1.014583e-07,5.718601e-07,9.050759e-09,1.363908e-06,...,6.399458e-08,1.192406e-05,1.726750e-07,4.245786e-07,7.344992e-09,9.518085e-08,3.721983e-06,1.655280e-07,7.924373e-08,1.346388e-08,2.316122e-08,3.365276e-07,7.175941e-09,2.341029e-08,2.931698e-07,4.458563e-07,4.940963e-07,5.056665e-08,1.374802e-09,3.273360e-07,3.663583e-08,4.873589e-08,1.310111e-08,3.499698e-08,1.246058e-07,3.984230e-08,4.606122e-08,1.491928e-07,3.323657e-08,1.932655e-07,3.734891e-07,6.034442e-06,2.155405e-06,7.129804e-08,2.934389e-07,2.287612e-07,9.990690e-08,1.429624e-08,3.311136e-06,5
1,1.599470e-07,3.906797e-11,2.611124e-08,4.064196e-10,4.452543e-09,1.559676e-11,4.972526e-09,1.128676e-08,6.200806e-08,2.412147e-11,1.394036e-10,1.329038e-07,5.137624e-10,8.279552e-09,1.188420e-10,9.656483e-10,8.200171e-09,1.910349e-07,2.513693e-10,8.487887e-12,5.074328e-08,6.328862e-08,3.335613e-10,3.125939e-09,5.824008e-06,5.274549e-10,1.272596e-09,3.108153e-11,1.872244e-11,2.755560e-09,1.352122e-11,1.971132e-10,2.678495e-08,1.448407e-13,2.235946e-10,2.789895e-08,4.235265e-12,6.888509e-11,3.043412e-11,6.588111e-07,...,2.626303e-08,2.340297e-08,2.103257e-08,1.307327e-09,1.417098e-11,1.249224e-07,4.277743e-08,3.850852e-11,2.678787e-11,1.204867e-09,8.096161e-11,9.343012e-10,7.310395e-11,6.644527e-09,1.110292e-08,7.676096e-09,6.318763e-09,4.333890e-07,3.187895e-10,1.132950e-09,7.599781e-11,6.993562e-10,4.900661e-11,2.321422e-10,2.172521e-10,4.990607e-09,3.726733e-11,2.938928e-10,8.118273e-11,4.764495e-10,5.354232e-09,1.404089e-08,8.922941e-09,1.483065e-08,1.883586e-09,6.913089e-13,5.489281e-09,6.146229e-08,1.360501e-08,3
2,9.814446e-12,5.079448e-11,6.235529e-11,8.781818e-12,1.523425e-08,5.529006e-09,3.581679e-11,1.797976e-09,2.404363e-12,3.565200e-10,1.625956e-10,1.388733e-09,1.279130e-14,2.570001e-09,7.621947e-13,9.070885e-09,8.670822e-08,4.177335e-09,1.613342e-11,5.425477e-10,1.173229e-07,6.194594e-12,2.442850e-12,2.046868e-11,2.615798e-05,1.569983e-09,4.316894e-11,9.201715e-10,4.459270e-09,1.570928e-07,3.530238e-09,3.392414e-09,1.478514e-07,5.315302e-11,9.314409e-10,1.319113e-10,8.159178e-09,8.325095e-11,5.756414e-08,2.713639e-07,...,2.638092e-09,1.128282e-06,2.782590e-08,3.406920e-09,8.423344e-14,3.819630e-10,2.047126e-07,8.704749e-10,4.329496e-10,5.441815e-12,3.985891e-12,2.106085e-09,9.646708e-12,2.574482e-11,1.641902e-08,3.397954e-11,3.187865e-08,2.727566e-09,2.866984e-12,4.230520e-09,2.377531e-11,1.252843e-09,2.054892e-10,2.970592e-10,2.413894e-11,1.682336e-09,2.736865e-12,1.566578e-10,9.644997e-12,2.777163e-10,6.348471e-10,6.057572e-09,2.243938e-10,2.039026e-07,1.918022e-09,7.359054e-12,2.286350e-10,2.364420e-08,1.850572e-10,2
3,2.105866e-08,4.769444e-08,6.389127e-08,9.991250e-10,5.190867e-08,1.804813e-07,3.957941e-07,8.896820e-07,5.276281e-09,5.607099e-07,5.218061e-10,1.756783e-09,1.089420e-09,1.321725e-06,9.052951e-09,6.051816e-08,1.539918e-08,5.993585e-05,2.339029e-08,3.270765e-10,1.373959e-07,3.652406e-06,1.009335e-08,2.073680e-08,2.661620e-06,8.050912e-08,1.688675e-07,8.782989e-10,1.702393e-07,9.422458e-08,1.410757e-08,1.834457e-09,1.690430e-08,1.087439e-11,1.065383e-08,3.130725e-08,4.235772e-07,1.581905e-07,3.870149e-08,5.947024e-08,...,1.202307e-07,9.834765e-06,4.092449e-09,1.850667e-06,5.476257e-10,9.9

In [0]:
#fits SVM on vectorized training dataset
clf2 = SVC(kernel='linear')
clf2.fit(train_df.drop(('label'),axis=1),np.array(train_df.label,dtype=int))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = clf2.predict(test_df.drop(('label'),axis=1))
printScore(y_pred,np.array(test_df.label,dtype=int))

Accuracy: 21.8%
